In [27]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import os
import math
import datetime
import time
import threading
from concurrent.futures import ThreadPoolExecutor


executor = ThreadPoolExecutor(max_workers=5)

p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

urdf_path = "custom_plane.urdf"

if not os.path.exists(urdf_path):
    raise FileNotFoundError(f"The URDF file does not exist at the specified path: {urdf_path}")
plane_id = p.loadURDF(urdf_path)

p.setGravity(0, 0, -10)

carpos = [0, 0, 0.1]
car = p.loadURDF("husky/husky.urdf", carpos[0], carpos[1], carpos[2])

numJoints = p.getNumJoints(car)
for joint in range(numJoints):
    print(p.getJointInfo(car, joint))

targetVel = 1  # rad/s
maxForce = 100  # Newton
p.setRealTimeSimulation(1)

obstacle_positions = [
    [0, 5, 0.5],
    [2, 5, 0.5],
    [-2, 5, 0.5],
    [4, 5, 0.5],
    [-4, 5, 0.5],
    [0, 7, 0.5],
    [2, 7, 0.5],
    [-2, 7, 0.5],
    [4, 7, 0.5],
    [-4, 7, 0.5]
]

for pos in obstacle_positions:
    p.loadURDF("cube.urdf", pos, [0, 0, 1.73, 0.7], useFixedBase=False, globalScaling=0.5)

frame_counter = 0
capture_interval = 120
running = True
captured_images = []
data_log = []
filepaths = []

def capture_image():
    view_matrix = p.computeViewMatrix(camera_pos, camera_target, [0, 0, 1])
    projection_matrix = p.computeProjectionMatrixFOV(90, 1, 0.01, 100)
    images = p.getCameraImage(640, 480, view_matrix, projection_matrix, renderer=p.ER_BULLET_HARDWARE_OPENGL)
    img = np.array(images[2], dtype=np.uint8)
    img = np.reshape(img, (480, 640, 4))

    filename = datetime.datetime.now().strftime("%Y%m%d-%H%M%S%f") + '.png'
    filepath = os.path.join('captured_images', filename)

    data_log_entry = {
        "image_filename": filename,
        "key_states": key_states
    }

    return img, filepath, data_log_entry


def handle_capture_result(future):
    img, filepath, data_log_entry = future.result()
    captured_images.append(img)
    filepaths.append(filepath)
    data_log.append(data_log_entry)


while running:
    keys = p.getKeyboardEvents()
    key_states = [0, 0, 0, 0]

    moving_forward = False
    moving_backward = False
    steering_left = False
    steering_right = False

    pos, orn = p.getBasePositionAndOrientation(car)
    pos = list(pos)
    orn = p.getEulerFromQuaternion(orn)

    front_cam = [0.345 * math.cos(orn[2]), 0.345 * math.sin(orn[2]), 0.4]
    camera_pos = [pos[i] + front_cam[i] for i in range(3)]
    camera_target = [pos[0] + math.cos(orn[2]), pos[1] + math.sin(orn[2]), 0.4]

    for k, v in keys.items():
        if k == p.B3G_UP_ARROW:
            if v & p.KEY_IS_DOWN:
                moving_forward = True
                key_states[0] = int(v & p.KEY_IS_DOWN)
            elif v & p.KEY_WAS_RELEASED:
                moving_forward = False

        if k == p.B3G_DOWN_ARROW:
            if v & p.KEY_IS_DOWN:
                moving_backward = True
                key_states[1] = int(v & p.KEY_IS_DOWN)
            elif v & p.KEY_WAS_RELEASED:
                moving_backward = False

        if k == p.B3G_LEFT_ARROW:
            if v & p.KEY_IS_DOWN:
                steering_left = True
                key_states[2] = int(v & p.KEY_IS_DOWN)
            elif v & p.KEY_WAS_RELEASED:
                steering_left = False

        if k == p.B3G_RIGHT_ARROW:
            if v & p.KEY_IS_DOWN:
                steering_right = True
                key_states[3] = int(v & p.KEY_IS_DOWN)
            elif v & p.KEY_WAS_RELEASED:
                steering_right = False

        if k == ord('s') and v & p.KEY_WAS_TRIGGERED:
            running = False

    # Apply forward and backward motion
    if moving_forward:
        for joint in range(2, 6):
            p.setJointMotorControl2(car, joint, p.VELOCITY_CONTROL, targetVelocity=targetVel, force=maxForce)
    elif moving_backward:
        for joint in range(2, 6):
            p.setJointMotorControl2(car, joint, p.VELOCITY_CONTROL, targetVelocity=-targetVel, force=maxForce)
    else:
        for joint in range(2, 6):
            p.setJointMotorControl2(car, joint, p.VELOCITY_CONTROL, targetVelocity=0, force=maxForce)

    # Apply steering
    if steering_left:
        if moving_forward or moving_backward:
            # Reduce speed on left wheels
            for joint in [2, 4]:  # Assuming these are the left side wheel joints
                p.setJointMotorControl2(car, joint, p.VELOCITY_CONTROL, targetVelocity=targetVel/2.5 if moving_forward else -targetVel/2.5, force=maxForce)
    elif steering_right:
        if moving_forward or moving_backward:
            # Reduce speed on right wheels
            for joint in [3, 5]:  # Assuming these are the right side wheel joints
                p.setJointMotorControl2(car, joint, p.VELOCITY_CONTROL, targetVelocity=targetVel/2.5 if moving_forward else -targetVel/2.5, force=maxForce)


    if frame_counter % capture_interval == 0:
        future = executor.submit(capture_image)
        future.add_done_callback(handle_capture_result)

    frame_counter += 1

    p.stepSimulation()


p.getContactPoints(car)
p.disconnect()

executor.shutdown()


(0, b'chassis_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'base_link', (0.0, 0.0, 0.0), (0.0, 0.0, 0.14493), (0.0, 0.0, 0.0, 1.0), -1)
(1, b'imu_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'imu_link', (0.0, 0.0, 0.0), (0.08748, 0.00085, 0.09053), (0.0, 0.0, 0.0, 1.0), 0)
(2, b'front_left_wheel', 0, 7, 6, 1, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'front_left_wheel_link', (0.0, 1.0, 0.0), (0.34348, 0.28625, -0.06665), (0.0, 0.0, 0.0, 1.0), 0)
(3, b'front_right_wheel', 0, 8, 7, 1, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'front_right_wheel_link', (0.0, 1.0, 0.0), (0.34348, -0.28454999999999997, -0.06665), (0.0, 0.0, 0.0, 1.0), 0)
(4, b'rear_left_wheel', 0, 9, 8, 1, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'rear_left_wheel_link', (0.0, 1.0, 0.0), (-0.16852, 0.28625, -0.06665), (0.0, 0.0, 0.0, 1.0), 0)
(5, b'rear_right_wheel', 0, 10, 9, 1, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'rear_right_wheel_link', (0.0, 1.0, 0.0), (-0.16852, -0.28454999999999997, -0.06665), (0.0, 0.0, 0.0, 1.0), 0)
(6, b'top_plate

error: Not connected to physics server.

In [24]:
from itertools import zip_longest
for image_cap, image_path in zip_longest(captured_images, filepaths):
    print(image_path)
    rgb_image = cv2.cvtColor(image_cap, cv2.COLOR_BGR2RGB)
    cv2.imwrite(image_path, rgb_image)

captured_images\20240526-113632378802.png
captured_images\20240526-113632614929.png
captured_images\20240526-113632835627.png
captured_images\20240526-113633036770.png
captured_images\20240526-113633272906.png
captured_images\20240526-113633507499.png
captured_images\20240526-113633696097.png
captured_images\20240526-113633918238.png
captured_images\20240526-113634182857.png
captured_images\20240526-113634404651.png
captured_images\20240526-113634671451.png
captured_images\20240526-113634890882.png
captured_images\20240526-113635112925.png
captured_images\20240526-113635381937.png
captured_images\20240526-113635631781.png
captured_images\20240526-113635852268.png
captured_images\20240526-113636118205.png
captured_images\20240526-113636337183.png
captured_images\20240526-113636591103.png
captured_images\20240526-113636826741.png
captured_images\20240526-113637048786.png
captured_images\20240526-113637331206.png
captured_images\20240526-113637799067.png
captured_images\20240526-113637697

In [25]:
print(len(captured_images))
print(len(filepaths))

73
73


In [26]:
import csv
file_path = 'data.csv'

file_exists = os.path.isfile(file_path)
file_is_empty = os.path.getsize(file_path) == 0 if file_exists else True


with open(file_path, 'a', newline='') as csvfile:
    fieldnames = ['image_filename', 'key_states']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    if not file_exists or file_is_empty:
        writer.writeheader()

    for row in data_log:
        writer.writerow(row)

print("CSV file created/appended successfully.")

CSV file created/appended successfully.
